In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def last(listo):
    return listo[len(listo)-1]

def differenceTable(listo):
    answer=[]
    for j in [1..len(listo)-1]:
        answer=answer+[listo[j]-listo[j-1]]
    return answer
    

In [26]:
print differenceTable([1,5,3])

[4, -2]


In [ ]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()# output snipped 

import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
# output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

121
[0, 24*x^3 + 32*x]


In [4]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

24*x^3 + 32*x
276*x^6 - 32*x^4 - 192*x^2
2048*x^9 - 237568/27*x^7 + 32768/27*x^5 + 131072/27*x^3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
49152*x^15 - 1072627712/1125*x^13 + 4173856768/675*x^11 - 45736787968/3375*x^9 + 1564475392/675*x^7 + 6257901568/1125*x^5
184024*x^18 - 144266176/27*x^16 + 43337949824/729*x^14 - 217405085696/729*x^12 + 140124878848/243*x^10 - 77900890112/729*x^8 - 155801780224/729*x^6


In [5]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print polynomialnumerator(D(poly))

24*x^3 + 32*x
276*x^6 - 32*x^4 - 192*x^2
55296*x^9 - 237568*x^7 + 32768*x^5 + 131072*x^3
11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
165888000*x^15 - 3217883136*x^13 + 20869283840*x^11 - 45736787968*x^9 + 7822376960*x^7 + 18773704704*x^5
134153496*x^18 - 3895186752*x^16 + 43337949824*x^14 - 217405085696*x^12 + 420374636544*x^10 - 77900890112*x^8 - 155801780224*x^6


In [2]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==0:data=data+[index]
print data

[]


In [3]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd>0:data=data+[sd]
print Set(data)

{1, 2, 4, 5, 6}


In [7]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=0
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)
        


(0, 0, 1, 3)
(7, 1, 4, 24)
(14, 2, 4, 43)
(21, 3, 4, 62)
(28, 4, 4, 81)
(35, 5, 4, 100)
(42, 6, 4, 119)
(49, 7, 4, 138)
(56, 8, 4, 157)
(63, 9, 4, 176)
(70, 10, 4, 195)
(77, 11, 4, 214)
(84, 12, 4, 233)
(91, 13, 4, 252)
(98, 14, 4, 271)
(105, 15, 4, 290)
(112, 16, 4, 309)
(119, 17, 4, 328)
[21, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


In [8]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=1
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(1, 0, 4, 6)
(8, 1, 4, 25)
(15, 2, 4, 44)
(22, 3, 4, 63)
(29, 4, 4, 82)
(36, 5, 4, 101)
(43, 6, 4, 120)
(50, 7, 4, 139)
(57, 8, 4, 158)
(64, 9, 4, 177)
(71, 10, 4, 196)
(78, 11, 4, 215)
(85, 12, 4, 234)
(92, 13, 4, 253)
(99, 14, 4, 272)
(106, 15, 4, 291)
(113, 16, 4, 310)
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


In [9]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=2
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(2, 0, 2, 9)
(9, 1, 2, 28)
(16, 2, 2, 47)
(23, 3, 2, 66)
(30, 4, 2, 85)
(37, 5, 2, 104)
(44, 6, 2, 123)
(51, 7, 2, 142)
(58, 8, 2, 161)
(65, 9, 2, 180)
(72, 10, 2, 199)
(79, 11, 2, 218)
(86, 12, 2, 237)
(93, 13, 2, 256)
(100, 14, 2, 275)
(107, 15, 2, 294)
(114, 16, 2, 313)
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


In [10]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(3, 0, 2, 12)
(10, 1, 2, 31)
(17, 2, 2, 50)
(24, 3, 2, 69)
(31, 4, 2, 88)
(38, 5, 2, 107)
(45, 6, 2, 126)
(52, 7, 2, 145)
(59, 8, 2, 164)
(66, 9, 2, 183)
(73, 10, 2, 202)
(80, 11, 2, 221)
(87, 12, 2, 240)
(94, 13, 2, 259)
(101, 14, 2, 278)
(108, 15, 2, 297)
(115, 16, 2, 316)
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


In [11]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=4
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(4, 0, 6, 15)
(11, 1, 6, 34)
(18, 2, 6, 53)
(25, 3, 6, 72)
(32, 4, 6, 91)
(39, 5, 6, 110)
(46, 6, 6, 129)
(53, 7, 6, 148)
(60, 8, 6, 167)
(67, 9, 6, 186)
(74, 10, 6, 205)
(81, 11, 6, 224)
(88, 12, 6, 243)
(95, 13, 6, 262)
(102, 14, 6, 281)
(109, 15, 6, 300)
(116, 16, 6, 319)
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


In [12]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=5
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(5, 0, 5, 18)
(12, 1, 5, 37)
(19, 2, 5, 56)
(26, 3, 5, 75)
(33, 4, 5, 94)
(40, 5, 5, 113)
(47, 6, 5, 132)
(54, 7, 5, 151)
(61, 8, 5, 170)
(68, 9, 5, 189)
(75, 10, 5, 208)
(82, 11, 5, 227)
(89, 12, 5, 246)
(96, 13, 5, 265)
(103, 14, 5, 284)
(110, 15, 5, 303)
(117, 16, 5, 322)
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


In [13]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=6
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        td=totalDegreeOverField(poly,prime,sd)
        l=(index-residue)/prime
        print (index,l,sd,td)
        data=data+[td]
print differenceTable(data)

(-1, -1, 1, 0)
(6, 0, 1, 19)
(13, 1, 1, 38)
(20, 2, 1, 57)
(27, 3, 1, 76)
(34, 4, 1, 95)
(41, 5, 1, 114)
(48, 6, 1, 133)
(55, 7, 1, 152)
(62, 8, 1, 171)
(69, 9, 1, 190)
(76, 10, 1, 209)
(83, 11, 1, 228)
(90, 12, 1, 247)
(97, 13, 1, 266)
(104, 14, 1, 285)
(111, 15, 1, 304)
(118, 16, 1, 323)
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
